In [4]:
import torch
import torch.nn as nn
import numpy as np
import random

import gc
import psutil
import os
import multiprocessing as mp

p = psutil.Process(os.getpid())
p.nice(10)

num_cores = mp.cpu_count()
torch.set_num_threads(num_cores)
torch.set_num_interop_threads(num_cores)

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

with open("studyText/dataset.txt") as t:
    text = t.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
print('Уникальные символы:', ''.join(chars))
print(f'Размер словаря: {vocab_size}', '\n')

char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}


def encode(s): return [char_to_idx[c] for c in s]
def decode(l): return ''.join([idx_to_char[i] for i in l])


Уникальные символы: 
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~©«¬¯°²´·»½¿Ë×Øéëìóö÷ĘęśźŻżƍǝɐɓɔɯʁʎʖʚʞ̴̵̸̡̢̧̨̛̖̗̘̙̜̝̞̟̠̣̤̥̦̩̪̫̬̭̮̯̰̱̲̹̺̻̼͇͈͉͍͎͓͔͕͖͙͚̀́̂̃̄̅̆̇̈̉̋̌̍̎̏̐̑̒̓̔̽̾̀́͂̓̈́͆͊͋͌͐͑͒͗͛ͣͥͧͪͬͮͯ̕̚͘͜͝͠͡ͅΒΗΜΠδεοπЁІЇАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёєії҉აბგდევზთიკლმნორსტუფქღყშჩცწხჯჰᲠ​‍‐–—‘’‚“”•…″⁃₂₃€№⇄−√≈Ⓘⓘ█▒☝☺☻♂⚡⚰✉❗❤⠁⠂⠅⠇⠊⠍⠏⠑⠕⠗⠚⠝⠥⠫⠱⠲⠵。あかくしせたとなにはまもをん为举事人代任会副卐名命已并应成手接数新最果然犯理由的直私经结给翻老虽表訳誰选长️﻿，－🇱🇵🌈🌚🌝🌲🍎🍏🍬🎅🎉🏪🏳🏻🏼🐌🐻👀👇👈👉👞👣👦👨👩💌💨💭💰💺🔞🔥🕺😂😃😅😈😉😊😋😍😎😏😓😜😭😱😳😹😻🙏🚗🤙🤝🤡🤣🤯🤵🤷🥀🥃🥺🫡
Размер словаря: 514 



In [5]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=2, dropout=0.2):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    
    def forward(self, x, hidden):
        embedded = self.embedding(x)
        
        output, hidden = self.rnn(embedded, hidden)

        output = self.fc(output)
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        h = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return (h, c)

hidden_size = 256
model = CharRNN(vocab_size, hidden_size)
print(model)

CharRNN(
  (embedding): Embedding(514, 256)
  (rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=514, bias=True)
)


In [6]:
data = encode(text)
data = torch.tensor(data, dtype=torch.long)


seq_length = 50
batch_size = 64

def get_batch():
    start_idxs = torch.randint(0, len(data) - seq_length, (batch_size,))
    
    x_batch = torch.stack([data[i:i+seq_length] for i in start_idxs])
    y_batch = torch.stack([data[i+1:i+seq_length+1] for i in start_idxs])
    
    return x_batch, y_batch

x, y = get_batch()
print("Входная последовательность:", x[0][:10], "...")
print("Целевая последовательность:", y[0][:10], "...")
print("Текст входа:", decode(x[0][:10].tolist()))
print("Текст цели:", decode(y[0][:10].tolist()))

Входная последовательность: tensor([  1, 251, 275, 273, 277, 289, 273, 291, 281, 283]) ...
Целевая последовательность: tensor([251, 275, 273, 277, 289, 273, 291, 281, 283,  14]) ...
Текст входа:  Квадратик
Текст цели: Квадратик-


In [9]:
learning_rate = 0.00005
num_epochs = 35000
print_interval = 1000

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=0.9)

model.train()
for epoch in range(num_epochs):
    x_batch, y_batch = get_batch()
    
    optimizer.zero_grad()

    hidden = model.init_hidden(batch_size)
    
    output, hidden = model(x_batch, hidden)
    
    loss = criterion(output.view(-1, vocab_size), y_batch.view(-1))
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    gc.collect()
    
    if epoch % print_interval == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
        

print("Обучение завершено!")

Epoch 0, Loss: 2.2820
Epoch 1000, Loss: 2.2730
Epoch 2000, Loss: 2.2660
Epoch 3000, Loss: 2.1493
Epoch 4000, Loss: 2.1545
Epoch 5000, Loss: 2.1103
Epoch 6000, Loss: 2.0846
Epoch 7000, Loss: 2.0856
Epoch 8000, Loss: 2.0345
Epoch 9000, Loss: 1.9786
Epoch 10000, Loss: 1.9909
Epoch 11000, Loss: 1.9681
Epoch 12000, Loss: 1.9593
Epoch 13000, Loss: 2.0277
Epoch 14000, Loss: 1.8486
Epoch 15000, Loss: 2.0682
Epoch 16000, Loss: 1.9132
Epoch 17000, Loss: 1.9814
Epoch 18000, Loss: 1.9395
Epoch 19000, Loss: 1.9039
Epoch 20000, Loss: 1.8731
Epoch 21000, Loss: 1.9040
Epoch 22000, Loss: 1.8487
Epoch 23000, Loss: 1.8687
Epoch 24000, Loss: 1.8641
Epoch 25000, Loss: 1.8713
Epoch 26000, Loss: 1.8327
Epoch 27000, Loss: 1.8783
Epoch 28000, Loss: 1.8024
Epoch 29000, Loss: 1.8873
Epoch 30000, Loss: 1.8973
Epoch 31000, Loss: 1.7953
Epoch 32000, Loss: 1.7558
Epoch 33000, Loss: 1.8347
Epoch 34000, Loss: 1.7498
Обучение завершено!


In [10]:
def generate_text(model, start_str, length=100, temperature=0.8):
    model.eval()
    
    chars = [ch for ch in start_str]
    input_seq = torch.tensor(encode(chars), dtype=torch.long).unsqueeze(0)
    hidden = model.init_hidden(1)
    
    for _ in range(length):
        with torch.no_grad():
            output, hidden = model(input_seq, hidden)
            
            logits = output[0, -1, :] / temperature
            probabilities = torch.softmax(logits, dim=0)
            
            next_char_idx = torch.multinomial(probabilities, 1).item()
            
            chars.append(idx_to_char[next_char_idx])
            input_seq = torch.tensor([[next_char_idx]], dtype=torch.long)
    
    return ''.join(chars)

model.eval()
generated_text = generate_text(model, "Трава", 100)
print("Сгенерированный текст:")
print(generated_text)

Сгенерированный текст:
Трава в сменят нерекоротное домой...
Фервый в друг спрашивает: - Ты как пожалуйста, что на пароне приходи


In [13]:
def chat_with_model(model, initial_prompt="Привет", response_length=50):
    """Функция для 'общения' с моделью"""
    print("Вы:", initial_prompt)
    
    response = generate_text(model, initial_prompt, response_length)

    generated_response = response[len(initial_prompt):]
    print("Модель:", generated_response)

chat_with_model(model, "как дела")

Вы: как дела
Модель: .
Голоса от русских все читаю получает тот, а не п


In [ ]:
def save_checkpoint(model, optimizer, epoch, loss, path='checkpoint.pth'):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, path)
    print(f'Checkpoint saved: {path}')

save_checkpoint(model, optimizer, epoch, loss, 'model_checkpoint.pth')

Checkpoint saved: model_checkpoint.pth


In [ ]:
def load_for_generation(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    
    model = CharRNN(
        vocab_size=checkpoint['model_config']['vocab_size'],
        hidden_size=checkpoint['model_config']['hidden_size'],
        num_layers=checkpoint['model_config']['num_layers'],
        dropout=checkpoint['model_config']['dropout']
    )
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    char_to_idx = checkpoint['char_to_idx']
    idx_to_char = checkpoint['idx_to_char']
    chars = checkpoint['chars']
    
    return model, char_to_idx, idx_to_char, chars

model, char_to_idx, idx_to_char, chars = load_for_generation('checkpoints/best_model.pth')